# Chapter 25: Architecture Patterns

As applications grow beyond simple CRUD operations into complex platforms serving millions of users, architectural decisions made early profoundly impact maintainability, scalability, and team velocity. Modern Next.js applications demand architecture patterns that accommodate Server Components, Client Components, API routes, and shared UI while maintaining clear boundaries and allowing independent feature development.

By the end of this chapter, you'll master Feature-Sliced Design for scalable feature organization, implement layered architecture for clear separation of concerns, configure monorepos with Turborepo for multi-application ecosystems, design micro-frontend architectures, integrate design systems effectively, establish scalable project structures, and implement team collaboration patterns that prevent merge conflicts and enable parallel development.

## 25.1 Feature-Sliced Design (FSD)

Feature-Sliced Design is an architectural methodology for frontend applications that structures code by business domains rather than technical layers.

### Core Concepts of FSD

FSD organizes code into layers, slices, and segments:

```text
src/
├── app/                    # Next.js app router (thin layer)
├── pages/                  # Next.js pages (if using)
├── widgets/               # Independent UI blocks
│   ├── header/
│   ├── sidebar/
│   └── post-card/
├── features/              # User interactions and business logic
│   ├── auth/
│   ├── create-post/
│   └── add-to-cart/
├── entities/              # Business entities and data models
│   ├── user/
│   ├── post/
│   └── product/
├── shared/                # Reusable infrastructure
│   ├── api/
│   ├── ui/               # Design system components
│   ├── config/
│   └── lib/
└── processes/             # Complex multi-page scenarios
    └── checkout/
```

### Implementing FSD in Next.js

Structure features with clear public APIs:

```typescript
// src/features/auth/index.ts
// Public API - only export what other features can use
export { AuthGuard } from './ui/auth-guard';
export { LoginForm } from './ui/login-form';
export { useAuth } from './model/use-auth';
export { authSlice } from './model/auth-slice';
export type { User, Session } from './model/types';

// src/features/auth/ui/login-form.tsx
'use client';

import { useState } from 'react';
import { useAuth } from '../model/use-auth';
import { Input } from '@/shared/ui/input';
import { Button } from '@/shared/ui/button';

export function LoginForm() {
  const { login, isLoading } = useAuth();
  const [email, setEmail] = useState('');
  
  return (
    <form onSubmit={() => login(email)}>
      <Input 
        value={email} 
        onChange={setEmail}
        placeholder="Email" 
      />
      <Button loading={isLoading}>Login</Button>
    </form>
  );
}

// src/features/auth/model/use-auth.ts
import { useCallback } from 'react';
import { create } from 'zustand';
import { api } from '@/shared/api';

interface AuthState {
  user: User | null;
  login: (email: string) => Promise<void>;
  logout: () => void;
}

export const useAuth = create<AuthState>((set) => ({
  user: null,
  login: async (email) => {
    const user = await api.auth.login(email);
    set({ user });
  },
  logout: () => set({ user: null }),
}));
```

### Cross-Feature Communication

FSD enforces unidirectional dependencies (features cannot import from widgets or other features arbitrarily):

```typescript
// ❌ Bad: Feature importing from another feature directly
// src/features/comments/ui/comment-form.tsx
import { useAuth } from '@/features/auth/model/use-auth'; // Violates FSD

// ✅ Good: Using shared or composition
// Pass auth state via props or context from the widget/app level
// src/features/comments/ui/comment-form.tsx
interface CommentFormProps {
  isAuthenticated: boolean;
  userId: string;
}

export function CommentForm({ isAuthenticated, userId }: CommentFormProps) {
  // Component logic
}

// src/widgets/post-detail/ui/post-detail.tsx
// Widget composes features
import { useAuth } from '@/features/auth';
import { CommentForm } from '@/features/comments';
import { PostContent } from '@/entities/post';

export function PostDetail({ postId }: { postId: string }) {
  const { user, isAuthenticated } = useAuth();
  
  return (
    <div>
      <PostContent id={postId} />
      {isAuthenticated && (
        <CommentForm userId={user.id} isAuthenticated={isAuthenticated} />
      )}
    </div>
  );
}
```

## 25.2 Layered Architecture

Separate concerns into distinct layers with clear responsibilities.

### The Layer Pattern

```typescript
// src/shared/api/base.ts
// Infrastructure layer - HTTP client configuration
import { createClient } from '@supabase/supabase-js';

export const apiClient = createClient(
  process.env.NEXT_PUBLIC_SUPABASE_URL!,
  process.env.NEXT_PUBLIC_SUPABASE_ANON_KEY!
);

// src/entities/post/api/post-api.ts
// Data access layer
import { apiClient } from '@/shared/api';

export const postApi = {
  getById: (id: string) => 
    apiClient.from('posts').select('*').eq('id', id).single(),
  
  getAll: (params: PaginationParams) =>
    apiClient.from('posts').select('*', { count: 'exact' })
      .range(params.start, params.end),
};

// src/entities/post/model/post.ts
// Domain layer - Business logic
export interface Post {
  id: string;
  title: string;
  content: string;
  status: 'draft' | 'published';
  publishedAt?: Date;
}

export function isPublished(post: Post): boolean {
  return post.status === 'published' && !!post.publishedAt;
}

export function getExcerpt(post: Post, maxLength: number = 200): string {
  return post.content.slice(0, maxLength) + '...';
}

// src/features/edit-post/model/use-edit-post.ts
// Application layer - Use cases
import { useMutation, useQueryClient } from '@tanstack/react-query';
import { postApi } from '@/entities/post/api';
import { Post } from '@/entities/post/model';

export function useEditPost() {
  const queryClient = useQueryClient();
  
  return useMutation({
    mutationFn: (post: Partial<Post>) => postApi.update(post.id!, post),
    onSuccess: (data) => {
      // Invalidate and refetch
      queryClient.invalidateQueries({ queryKey: ['post', data.id] });
    },
  });
}

// src/features/edit-post/ui/edit-post-form.tsx
// Presentation layer
'use client';

import { useEditPost } from '../model/use-edit-post';
import { Button } from '@/shared/ui/button';

export function EditPostForm({ post }: { post: Post }) {
  const { mutate, isPending } = useEditPost();
  
  return (
    <form onSubmit={(e) => {
      e.preventDefault();
      mutate(formData);
    }}>
      {/* Form fields */}
      <Button loading={isPending}>Save Changes</Button>
    </form>
  );
}
```

## 25.3 Monorepo Setup

Scale to multiple applications and packages with Turborepo.

### Turborepo Configuration

```json
// turbo.json
{
  "$schema": "https://turbo.build/schema.json",
  "globalDependencies": ["**/.env.*local"],
  "pipeline": {
    "build": {
      "dependsOn": ["^build"],
      "outputs": [".next/**", "!.next/cache/**"]
    },
    "lint": {
      "outputs": []
    },
    "type-check": {
      "outputs": []
    },
    "dev": {
      "cache": false,
      "persistent": true
    },
    "test": {
      "dependsOn": ["build"]
    }
  }
}
```

```json
// package.json (root)
{
  "name": "my-monorepo",
  "private": true,
  "scripts": {
    "build": "turbo run build",
    "dev": "turbo run dev",
    "lint": "turbo run lint",
    "type-check": "turbo run type-check",
    "test": "turbo run test"
  },
  "devDependencies": {
    "turbo": "^1.10.0"
  },
  "workspaces": ["apps/*", "packages/*"]
}
```

### Package Structure

```text
my-monorepo/
├── apps/
│   ├── web/                    # Main Next.js app
│   │   ├── app/
│   │   ├── package.json
│   │   └── next.config.js
│   ├── admin/                  # Admin dashboard
│   └── api/                    # Standalone API (if needed)
├── packages/
│   ├── ui/                     # Shared UI components
│   │   ├── src/
│   │   ├── package.json
│   │   └── tsconfig.json
│   ├── config/                 # Shared config (eslint, ts)
│   ├── database/               # Prisma schema and client
│   └── utils/                  # Shared utilities
└── turbo.json
```

### Shared UI Package

```typescript
// packages/ui/src/components/button.tsx
import { cva, type VariantProps } from 'class-variance-authority';
import { cn } from '../utils';

const buttonVariants = cva(
  'inline-flex items-center justify-center rounded-md text-sm font-medium',
  {
    variants: {
      variant: {
        default: 'bg-blue-600 text-white hover:bg-blue-700',
        destructive: 'bg-red-600 text-white hover:bg-red-700',
        outline: 'border border-gray-300 bg-transparent',
      },
      size: {
        default: 'h-10 px-4 py-2',
        sm: 'h-8 px-3',
        lg: 'h-12 px-8',
      },
    },
  }
);

export interface ButtonProps
  extends React.ButtonHTMLAttributes<HTMLButtonElement>,
    VariantProps<typeof buttonVariants> {
  loading?: boolean;
}

export function Button({ 
  className, 
  variant, 
  size, 
  loading,
  children,
  ...props 
}: ButtonProps) {
  return (
    <button
      className={cn(buttonVariants({ variant, size }), className)}
      disabled={loading || props.disabled}
      {...props}
    >
      {loading ? <Spinner /> : children}
    </button>
  );
}

// packages/ui/package.json
{
  "name": "@repo/ui",
  "version": "0.0.0",
  "main": "./src/index.ts",
  "types": "./src/index.ts",
  "dependencies": {
    "class-variance-authority": "^0.7.0",
    "clsx": "^2.0.0",
    "tailwind-merge": "^2.0.0"
  },
  "devDependencies": {
    "@repo/typescript-config": "*",
    "@types/react": "^18",
    "react": "^18",
    "typescript": "^5"
  }
}
```

### Next.js App Configuration

```javascript
// apps/web/next.config.js
module.exports = {
  transpilePackages: ['@repo/ui', '@repo/database', '@repo/utils'],
  experimental: {
    // Enable if using server components from external packages
    serverComponentsExternalPackages: ['@repo/database'],
  },
};
```

## 25.4 Micro-Frontends

Split large applications into independently deployable units.

### Module Federation with Next.js

```javascript
// next.config.js (Host app)
const { NextFederationPlugin } = require('@module-federation/nextjs-mf');

module.exports = {
  webpack(config, options) {
    if (!options.isServer) {
      config.plugins.push(
        new NextFederationPlugin({
          name: 'host',
          filename: 'static/chunks/remoteEntry.js',
          remotes: {
            shop: 'shop@https://shop.example.com/_next/static/chunks/remoteEntry.js',
            admin: 'admin@https://admin.example.com/_next/static/chunks/remoteEntry.js',
          },
          shared: {
            react: { singleton: true, requiredVersion: false },
            'react-dom': { singleton: true, requiredVersion: false },
            'next-auth': { singleton: true },
          },
        })
      );
    }
    return config;
  },
};
```

### Consuming Remote Components

```typescript
// apps/web/pages/index.tsx
import dynamic from 'next/dynamic';

// Lazy load remote component
const RemoteProductCard = dynamic(
  () => import('shop/product-card').then((mod) => mod.ProductCard),
  { 
    ssr: false,
    loading: () => <div>Loading product...</div>
  }
);

export default function HomePage() {
  return (
    <div>
      <h1>Welcome to our Store</h1>
      <RemoteProductCard sku="12345" />
    </div>
  );
}
```

### Multi-Zone Architecture

Simpler alternative to Module Federation using routing:

```javascript
// apps/web/next.config.js (Main zone)
module.exports = {
  async rewrites() {
    return [
      {
        source: '/shop',
        destination: 'https://shop-zone.example.com/shop',
      },
      {
        source: '/shop/:path*',
        destination: 'https://shop-zone.example.com/shop/:path*',
      },
      {
        source: '/admin',
        destination: 'https://admin-zone.example.com/admin',
      },
      {
        source: '/admin/:path*',
        destination: 'https://admin-zone.example.com/admin/:path*',
      },
    ];
  },
  
  // Shared session configuration
  async headers() {
    return [
      {
        source: '/:path*',
        headers: [
          {
            key: 'X-Zone',
            value: 'main',
          },
        ],
      },
    ];
  },
};
```

## 25.5 Design Systems Integration

Integrate and maintain design systems across applications.

### Component Library Architecture

```typescript
// packages/ui/src/theme/tokens.ts
export const tokens = {
  colors: {
    primary: {
      50: '#eff6ff',
      500: '#3b82f6',
      600: '#2563eb',
      900: '#1e3a8a',
    },
    // semantic colors
    background: '#ffffff',
    foreground: '#0f172a',
  },
  spacing: {
    xs: '0.25rem',
    sm: '0.5rem',
    md: '1rem',
    lg: '1.5rem',
    xl: '2rem',
  },
  radius: {
    sm: '0.25rem',
    md: '0.5rem',
    lg: '0.75rem',
    full: '9999px',
  },
};

// packages/ui/src/theme/provider.tsx
'use client';

import { createContext, useContext } from 'react';

const ThemeContext = createContext(tokens);

export function ThemeProvider({ 
  children, 
  customTokens 
}: { 
  children: React.ReactNode;
  customTokens?: Partial<typeof tokens>;
}) {
  const mergedTokens = deepMerge(tokens, customTokens || {});
  
  return (
    <ThemeContext.Provider value={mergedTokens}>
      {children}
    </ThemeContext.Provider>
  );
}

export const useTheme = () => useContext(ThemeContext);
```

### Tailwind Configuration Sharing

```javascript
// packages/tailwind-config/tailwind.config.js
const { fontFamily } = require('tailwindcss/defaultTheme');

module.exports = {
  darkMode: ['class'],
  content: [
    'app/**/*.{ts,tsx}',
    'components/**/*.{ts,tsx}',
    '../../packages/ui/src/**/*.{ts,tsx}',
  ],
  theme: {
    container: {
      center: true,
      padding: '2rem',
      screens: {
        '2xl': '1400px',
      },
    },
    extend: {
      colors: {
        border: 'hsl(var(--border))',
        input: 'hsl(var(--input))',
        ring: 'hsl(var(--ring))',
        background: 'hsl(var(--background))',
        foreground: 'hsl(var(--foreground))',
        primary: {
          DEFAULT: 'hsl(var(--primary))',
          foreground: 'hsl(var(--primary-foreground))',
        },
        // ... more tokens
      },
      borderRadius: {
        lg: 'var(--radius)',
        md: 'calc(var(--radius) - 2px)',
        sm: 'calc(var(--radius) - 4px)',
      },
      fontFamily: {
        sans: ['var(--font-sans)', ...fontFamily.sans],
      },
    },
  },
  plugins: [require('tailwindcss-animate')],
};

// apps/web/tailwind.config.js
const sharedConfig = require('@repo/tailwind-config/tailwind.config.js');

module.exports = {
  ...sharedConfig,
  content: [
    ...sharedConfig.content,
    './src/**/*.{js,ts,jsx,tsx}',
  ],
};
```

## 25.6 Scalable Project Structures

Organize code to scale with team size and feature count.

### Directory Organization Patterns

```text
src/
├── app/                          # Next.js routing (thin layer)
│   ├── (marketing)/
│   ├── (dashboard)/
│   ├── api/
│   └── layout.tsx               # Root layout only
├── features/                     # Business features
│   ├── feature-name/
│   │   ├── api/                 # API routes (if feature-specific)
│   │   ├── components/          # Feature components
│   │   ├── hooks/               # Feature hooks
│   │   ├── lib/                 # Feature utilities
│   │   ├── stores/              # State management
│   │   ├── types/               # TypeScript types
│   │   └── index.ts             # Public API
│   └── index.ts                 # Feature registry
├── shared/                       # Shared code
│   ├── components/              # Primitive UI
│   ├── hooks/                   # Shared hooks
│   ├── lib/                     # Utilities
│   ├── types/                   # Global types
│   └── constants.ts
└── middleware.ts                # Root middleware only
```

### Co-Location Strategy

Keep related files close together:

```typescript
// src/features/checkout/components/checkout-form.tsx
import { useCheckout } from '../hooks/use-checkout';
import { CheckoutSchema } from '../lib/schema';
import type { CheckoutData } from '../types';

// src/features/checkout/hooks/use-checkout.ts
import { useMutation } from '@tanstack/react-query';
import { checkoutApi } from '../api/checkout';

// src/features/checkout/api/checkout.ts
import { apiClient } from '@/shared/lib/api';

// src/features/checkout/lib/schema.ts
import { z } from 'zod';

export const CheckoutSchema = z.object({
  // validation
});

// src/features/checkout/types/index.ts
export interface CheckoutData {
  // types
}
```

### Route Groups for Organization

Use Next.js route groups for clean structure without URL impact:

```text
app/
├── (auth)/                      # Auth layout group
│   ├── layout.tsx
│   ├── login/
│   ├── register/
│   └── forgot-password/
├── (main)/                      # Main app layout
│   ├── layout.tsx
│   ├── page.tsx                 # Homepage
│   ├── dashboard/
│   └── settings/
├── (marketing)/                 # Marketing pages
│   ├── layout.tsx
│   ├── about/
│   ├── pricing/
│   └── blog/
└── api/                         # API routes outside groups
    ├── auth/
    └── webhooks/
```

## 25.7 Team Collaboration Patterns

Prevent conflicts and enable parallel development.

### API Contracts with TypeScript

Define shared types for frontend-backend contracts:

```typescript
// packages/api-contracts/src/user.ts
export interface CreateUserRequest {
  email: string;
  name: string;
  role: 'user' | 'admin';
}

export interface UserResponse {
  id: string;
  email: string;
  name: string;
  role: string;
  createdAt: string;
}

export interface UserListResponse {
  users: UserResponse[];
  total: number;
  page: number;
}

// Type-safe API client generation
// packages/api-client/src/index.ts
import type { CreateUserRequest, UserResponse } from '@repo/api-contracts';

export const userApi = {
  create: (data: CreateUserRequest) => 
    fetch('/api/users', {
      method: 'POST',
      body: JSON.stringify(data),
    }).then(r => r.json() as Promise<UserResponse>),
    
  list: (params: { page: number; limit: number }) =>
    fetch(`/api/users?page=${params.page}&limit=${params.limit}`)
      .then(r => r.json()),
};
```

### Feature Flags for Continuous Deployment

Deploy incomplete features safely:

```typescript
// src/shared/lib/features.ts
import { createClient } from '@vercel/edge-config';

const edgeConfig = createClient(process.env.EDGE_CONFIG);

export async function getFeatureFlags() {
  return {
    newDashboard: await edgeConfig.get('feature_new_dashboard'),
    betaCheckout: await edgeConfig.get('feature_beta_checkout'),
    darkMode: await edgeConfig.get('feature_dark_mode'),
  };
}

// Usage in Server Component
// app/page.tsx
export default async function Page() {
  const flags = await getFeatureFlags();
  
  return (
    <div>
      {flags.newDashboard ? <NewDashboard /> : <OldDashboard />}
    </div>
  );
}

// Usage in Client Component with middleware
// middleware.ts
export async function middleware(request: NextRequest) {
  const flags = await getFeatureFlags();
  
  // Redirect beta users
  if (request.nextUrl.pathname === '/checkout' && flags.betaCheckout) {
    return NextResponse.rewrite(new URL('/checkout-v2', request.url));
  }
  
  return NextResponse.next();
}
```

### Code Ownership with CODEOWNERS

```text
# .github/CODEOWNERS

# Global owners
* @team-leads

# Feature ownership
/src/features/checkout/ @checkout-team
/src/features/auth/ @platform-team

# Shared code requires broader review
/src/shared/api/ @platform-team @backend-team
/src/shared/ui/ @design-system-team

# Database changes require DBA review
/packages/database/ @dba-team
```

### Git Workflow for Next.js

```bash
# Feature branch workflow
git checkout -b feature/checkout-redesign
# ... work on feature ...

# Build verification before push
npm run build
npm run type-check
npm run test

# Push and create PR
git push origin feature/checkout-redesign

# After merge, clean up
git checkout main
git pull origin main
git branch -d feature/checkout-redesign
```

## Key Takeaways from Chapter 25

1. **Feature-Sliced Design (FSD)**: Organize code by business domains (widgets, features, entities, shared) rather than technical layers. Features encapsulate UI, state, and API logic with a public API exported from `index.ts`. Enforce import rules where upper layers cannot import from lower layers (entities cannot import from features).

2. **Layered Architecture**: Separate infrastructure (API clients), domain (business logic), application (use cases), and presentation (components). This allows swapping implementations (e.g., changing from REST to GraphQL) without affecting UI components.

3. **Monorepo Setup**: Use Turborepo to orchestrate builds across multiple apps and packages. Share UI components, database schemas, and TypeScript configurations through internal packages (`@repo/ui`, `@repo/database`). Configure `transpilePackages` in Next.js to consume TypeScript packages directly.

4. **Micro-Frontends**: Deploy independent applications using Multi-Zone architecture (rewrites/proxy) for simpler setups, or Module Federation for runtime integration. Ensure shared dependencies (React, NextAuth) are singletons to prevent version conflicts.

5. **Design Systems**: Create a shared UI package with design tokens (colors, spacing, typography) and primitive components. Use Tailwind CSS configuration sharing to maintain consistency across applications while allowing app-specific customization.

6. **Scalable Structure**: Co-locate related files (components, hooks, types, API calls) within feature directories. Use Next.js Route Groups `(marketing)`, `(dashboard)` to organize layouts without affecting URL structure. Keep the `app` directory thin—move business logic to feature folders.

7. **Team Collaboration**: Define API contracts in shared packages for type-safe communication between frontend and backend. Use feature flags (Vercel Edge Config, LaunchDarkly) to deploy incomplete features safely. Establish CODEOWNERS for automatic reviewer assignment based on file paths.

## Coming Up Next

**Chapter 26: Advanced Component Patterns**

Now that your application architecture is solid, it's time to master sophisticated component design. In Chapter 26, we'll explore Compound Components, Render Props, Higher-Order Components, Custom Hooks development, Headless UI patterns, Server Component composition strategies, and performance-optimized component patterns. You'll learn how to build flexible, reusable components that adapt to complex requirements while maintaining clean APIs.